2D Omnipose segmentation
============================

Before running this notebook, install the latest version of Omnipose from GitHub. This automatically installs our Cellpose fork. 

In [11]:
# make local editable packages automatically reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Import dependencies
import numpy as np
from cellpose_omni import models, core

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? {}'.format(use_GPU))

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
%matplotlib inline

2023-11-13 20:52:13,396 [INFO] TORCH GPU version not installed/working.
>>> GPU activated? False


In [15]:
from pathlib import Path
import os
from cellpose_omni import io
import omnipose

# Set the path to the desired folder
basedir = 'PilG_dilute_PC_contr5'

# Get the list of image files from the folder
files = io.get_image_files(basedir)

In [16]:
from cellpose_omni import io, transforms
from omnipose.utils import normalize99
imgs = [io.imread(f) for f in files]

#print some info about the images.
#for i in imgs:
#    print('Original image shape:',i.shape)
#    print('data type:',i.dtype)
#    print('data range: min {}, max {}\n'.format(i.min(),i.max()))
nimg = len(imgs)
print('\nnumber of images:',nimg)

#fig = plt.figure(figsize=[40]*2,frameon=False) # initialize figure
#print('\n')
#for k in range(len(imgs)):
#    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
#    if len(img.shape)>2:
#        imgs[k] = img[:,:,1] # could pick out a specific channel
#        imgs[k] = np.mean(img,axis=-1) # or just turn into grayscale 
        
#    imgs[k] = normalize99(imgs[k])
#    imgs[k] = np.pad(imgs[k],10,'edge')
#    print('new shape: ', imgs[k].shape)
#    plt.subplot(1,len(files),k+1)
#    plt.imshow(imgs[k],cmap='gray')
#    plt.axis('off')


number of images: 241


## Initialize model
Here we use one of the built-in model names. We will choose the `bact_phase_omni` model. 

In [17]:
from cellpose_omni import models
from cellpose_omni.models import MODEL_NAMES

model_name = 'bact_phase_omni'
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)

2023-11-13 20:53:58,281 [INFO] >>bact_phase_omni<< model set to be used
2023-11-13 20:53:58,282 [INFO] >>>> using CPU


## Run segmentation 

In [ ]:
import time
from tqdm import tqdm
chans = [0,0] #this means segment based on first channel, no second channel 

#n = [1] # make a list of integers to select which images you want to segment
n = range(nimg) # or just segment them all 

# define parameters
params = {'channels':chans, # always define this with the model
          'rescale': None, # upscale or downscale your images, None = no rescaling 
          'mask_threshold': 2.55, # erode or dilate masks with higher or lower values 
          'flow_threshold': 0, # default is .4, but only needed if there are spurious masks to clean up; slows down output
          'transparency': True, # transparency in flow output
          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 
          'cluster': True, # use DBSCAN clustering
          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 
          # 'verbose': False, # turn on if you want to see more output 
          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 
          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 
          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 
          'affinity_seg': False, # new feature, stay tuned...
         }


total_segmentation_time = 0

# Loop through each image, segment and save
for i in tqdm(n, 'Segmentation progress:'):
    tic = time.time()  # start time for this iteration
    
    # Perform segmentation on the i-th image
    mask, flow, style = model.eval(imgs[i], **params)
    
    # Calculate segmentation time for this image
    seg_time = time.time() - tic
    total_segmentation_time += seg_time  # add to total time
    
    print(f'Segmentation time for image {i}: {seg_time}s')
    
    # Define the file path for saving the results
    file_path = files[i]  # Assuming 'files' is your list of file paths for the images
    
    # Save the masks for the i-th image
    io.save_masks([imgs[i]], [mask], [flow], [file_path], 
                  png=False,
                  tif=True,
                  suffix='',  # Add suffix if needed
                  save_flows=False,
                  save_outlines=False,
                  dir_above=0,
                  in_folders=True,
                  save_txt=False,
                  save_ncolor=False)
    
print(f'Total segmentation time for all images: {total_segmentation_time}s')

average_time_per_image = total_segmentation_time / nimg
expected_total_time = average_time_per_image * nimg
print(f'Expected total time for segmenting all images: {expected_total_time}s')